In [1]:
import os
PATH = "/home/mdobrzanski/playground/IET/NLP/ustawy/"
%config Completer.use_jedi = False

In [47]:
import re
import time
from collections import defaultdict, Counter, OrderedDict
from functools import reduce
import math
import pandas as pd
import requests
import collections
import tensorflow as tf
import torch as pytorch
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, confusion_matrix, precision_score, recall_score
from imblearn.over_sampling import RandomOverSampler
import fasttext
from transformers import TFBertForSequenceClassification
from lime.lime_text import LimeTextExplainer

In [44]:
pip install lime

     |████████████████████████████████| 275 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 60.7 MB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 25.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 24.9 MB/s eta 0:00:01
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283845 sha256=d1ad1b72620ecce7f9f6ba0eb6846a01d146057bcee19bfa73ba922dbb60c251
  Stored in directory: /home/mdobrzanski/.cache/pip/wheels/e6/a6/20/cc1e293fcdb67ede666fed293cb895395e7ecceb4467779546
Successfully built lime
Note: you may need to restart the kernel to use updated packages.


In [3]:
from simpletransformers.classification import ClassificationModel

In [4]:
pytorch.cuda.is_available()

True

In [5]:
# pytorch.cuda.get_device_name(0)

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
# pip install torch==1.8.0

In [8]:
from ipywidgets import IntProgress

# 1

In [9]:
dataset_1 = load_dataset("poleval2019_cyberbullying", 'task01')

Reusing dataset poleval2019_cyber_bullying (/home/mdobrzanski/.cache/huggingface/datasets/poleval2019_cyber_bullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
dataset_2 = load_dataset("poleval2019_cyberbullying", 'task02')

Reusing dataset poleval2019_cyber_bullying (/home/mdobrzanski/.cache/huggingface/datasets/poleval2019_cyber_bullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
dataset_1

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10041
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [12]:
dataset_2

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10041
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

### Dataset 1

In [13]:
print('Train O: ', dataset_1['train']['label'].count(0))
print('Train 1: ', dataset_1['train']['label'].count(1))
print('Ratio train: ', dataset_1['train']['label'].count(1)/len(dataset_1['train']['label']))

Train O:  9190
Train 1:  851
Ratio train:  0.08475251468977194


In [14]:
print('Test O: ', dataset_1['test']['label'].count(0))
print('Test 1: ', dataset_1['test']['label'].count(1))
print('Ratio test: ', dataset_1['test']['label'].count(1)/len(dataset_1['test']['label']))

Test O:  866
Test 1:  134
Ratio test:  0.134


### Dataset 2

In [15]:
print('Train O: ', dataset_2['train']['label'].count(0))
print('Train 1: ', dataset_2['train']['label'].count(1))
print('Train 2: ', dataset_2['train']['label'].count(2))

Train O:  9190
Train 1:  253
Train 2:  598


In [16]:
print('Test O: ', dataset_2['test']['label'].count(0))
print('Test 1: ', dataset_2['test']['label'].count(1))
print('Test 2: ', dataset_2['test']['label'].count(2))

Test O:  866
Test 1:  25
Test 2:  109


### Prepare dataset 1

In [17]:
X1_tr_base = dataset_1['train']['text']
y1_tr = dataset_1['train']['label']

X1_tst_base = dataset_1['test']['text']
y1_tst = dataset_1['test']['label']

In [18]:
vect1 = TfidfVectorizer()

vect1.fit(X1_tr_base)

TfidfVectorizer()

In [19]:
X1_tr = vect1.transform(X1_tr_base)
X1_tst = vect1.transform(X1_tst_base)

In [20]:
ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
X1_tr, y1_tr = ros.fit_resample(X1_tr, y1_tr)

### Prepare dataset 2

In [21]:
X2_tr_base = dataset_2['train']['text']
y2_tr = dataset_2['train']['label']

X2_tst_base = dataset_2['test']['text']
y2_tst = dataset_2['test']['label']

In [22]:
vect2 = TfidfVectorizer()

vect2.fit(X2_tr_base)

TfidfVectorizer()

In [23]:
X2_tr = vect2.transform(X2_tr_base)
X2_tst = vect2.transform(X2_tst_base)

In [24]:
ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
X2_tr, y2_tr = ros.fit_resample(X2_tr, y2_tr)

# 2

### Bayesian classifier with TF * IDF weighting.

Dataset 1

In [25]:
bayes1 = MultinomialNB()
bayes1.fit(X1_tr, y1_tr)

MultinomialNB()

In [26]:
bayes1.score(X1_tst, y1_tst)

0.826

Dataset 2

In [27]:
bayes2 = MultinomialNB()
bayes2.fit(X2_tr, y2_tr)

MultinomialNB()

In [28]:
bayes2.score(X2_tst, y2_tst)

0.801

### Fasttext text classifier

In [29]:
with open('data_1_tr.txt', "w") as f:
    for text, y in zip(X1_tr, y1_tr):
        f.write("__label__{} {}\n".format(y, text))

with open('data_1_tst.txt', "w") as f:
    for text, y in zip(X1_tst, y1_tst):
        f.write("__label__{} {}\n".format(y, text))
        
with open('data_2_tr.txt', "w") as f:
    for text, y in zip(X2_tr, y2_tr):
        f.write("__label__{} {}\n".format(y, text))
        
with open('data_2_tst.txt', "w") as f:
    for text, y in zip(X2_tst, y2_tst):
        f.write("__label__{} {}\n".format(y, text))

In [30]:
fasttext1 = fasttext.train_supervised('data_1_tr.txt')

In [31]:
fasttext1.test('data_1_tst.txt')

(1000, 0.874, 0.874)

In [32]:
fasttext2 = fasttext.train_supervised('data_2_tr.txt')

In [33]:
fasttext2.test('data_2_tst.txt')

(1000, 0.864, 0.864)

### Transformer classifier 

In [34]:
model = ClassificationModel('bert', 'bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [35]:
tr_df1 = pd.DataFrame(list(map(list, zip(*[X1_tr_base, y1_tr]))), columns=['text', 'label'])

In [38]:
# model.train_model(tr_df1)

In [42]:
# model.predict(tr_df1['text'][0])

# 3

# 4

# 5